In [1]:
import torch
import os
import argparse
import pandas as pd

/home/ubuntu/anaconda3/envs/jeonghyeon/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from module.model import model as tagger
from module.model import device
from module.model import tokenizer
from module.model import id2label

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
# parser = argparse.ArgumentParser()
# parser.add_argument("-q", "--query", dest="keyword", action="store")
# args = parser.parse_args()

## Loading

In [4]:
file_list = os.listdir('./inference_data')
file_path = []
for file_name in file_list:
    tmp_path = os.path.join('./inference_data', file_name)
    file_path.append(tmp_path)
file_path.sort()

In [5]:
load_df = lambda x: pd.read_csv(x)
dfs = [load_df(x) for x in file_path]
data = dfs[0]
data.dropna(inplace=True)
# data[pd.isna(data).any(axis=1)]
# data.isna().nunique()
print(f'length of {file_path[0]}: {len(data)}')

length of ./inference_data/event_port.csv: 171


In [6]:
data.isnull().any()

Event    False
dtype: bool

## Cleansing

In [7]:
# def preprocess(df):
#     df['Event'] = df['Event'].str.replace("．", ".", regex=False)
#     df['Event'] = df['Event'].astype(str)
#     # df['Event'] = df['Event'].str.replace(r'[^ㄱ-ㅣ가-힣0-9a-zA-Z.]+', " ", regex=True)
#     return df

In [8]:
# data = preprocess(data)

In [9]:
keyword = '화물'

data['Indexes'] = data.Event.str.find(keyword)
infer_data = data[data.Indexes > -1]
# infer_data

In [10]:
# for event in infer_data.Event:
#     print(event)
#     print()

In [11]:
infer_data_lst = infer_data.Event.apply(lambda x: tokenizer(x, max_length=256, padding='max_length', truncation=True))
input_ids_lst = []
input_masks_lst = []
for e in infer_data_lst:
    input_ids_lst.append(e['input_ids'])
    input_masks_lst.append(e['attention_mask'])
len(input_ids_lst), len(input_masks_lst)

(36, 36)

In [12]:
def infer(input_ids_lst, input_masks_lst):

    tagger.eval()
    with torch.no_grad():
        infer_results = []
        for input_ids, input_masks in zip(input_ids_lst, input_masks_lst): 
            input_tensor = torch.LongTensor(input_ids).unsqueeze(0).to(device)
            mask_tensor = torch.LongTensor(input_masks).unsqueeze(0).to(device)
            output = tagger(input_tensor, mask_tensor)
            pred = torch.argmax(output, dim=-1).squeeze().detach().cpu().tolist()
            infer_results.append(pred)

        return infer_results
    
infer_results = infer(input_ids_lst, input_masks_lst)

In [18]:
subwordsList = []
tagsList = []
for input_ids, result in zip(input_ids_lst, infer_results):
    decoded = tokenizer.decode(input_ids, skip_special_tokens=True)
    subwords = tokenizer.encode(decoded)[1:-1]
    subwords = list(map(lambda x: tokenizer.convert_ids_to_tokens(x), subwords))
    subwordsList.append(subwords)
    lenSubwords = len(subwords)    

    result = result[1:lenSubwords+1]
    tags = []
    for id in result:
        tags.append(id2label[id])
    tagsList.append(tags)

In [ ]:
# 날짜(DAT), 시간(TIME), 장소(LOC), 작업 정보(WRK)를 추출해서 txt 파일에 저장

def extract(tagtype, tags, subwords):
    # if tag does not exists then notify and pass
    # if not '{tag}_I' then stop
    tag_start = f'{tagtype}_B'
    if tag_start in tags:
        b_idx = tags.index(tag_start)
        for i, tag in enumerate(tags[b_idx:]):
            if i > 0 and tag != f'{tagtype}_I':
                # print(tags[b_idx:b_idx+i+1])
                # print(tags[b_idx:b_idx+i])
                tagged = subwords[b_idx:b_idx+i]
                tagged = tokenizer.convert_tokens_to_string(tagged)
                # print(tagged)
                return tagged
    else:
        msg = f'{tagtype} not found.'
        # print(msg)
        return msg

tagtypes = dict(
    dat='DAT',
    tim='TIM',
    loc='LOC',
    wrk='WRK',    
)

# subwordsList, tagsList
nerResults = []
count = 0
for subwords, tags in zip(subwordsList, tagsList):
    # print(f'index: {count}')
    extracted = []
    for tagtype in tagtypes.values():
        # print(tagtype)
        tagged = extract(tagtype, tags, subwords)
        extracted.append(tagged)
        # print()
    nerResults.append(extracted)
    count += 1

In [19]:
# v2

# 날짜(DAT), 시간(TIME), 장소(LOC), 작업 정보(WRK)를 추출해서 txt 파일에 저장

def extract(tagtype, tags, subwords):
    # if tag does not exists then notify and pass
    # if not '{tag}_I' then stop
    tag_start = f'{tagtype}_B'
    if tag_start in tags:
        b_idx = tags.index(tag_start)
        for i, tag in enumerate(tags[b_idx:]):
            if i > 0 and tag != f'{tagtype}_I':
                # print(tags[b_idx:b_idx+i+1])
                # print(tags[b_idx:b_idx+i])
                tagged = subwords[b_idx:b_idx+i]
                tagged = tokenizer.convert_tokens_to_string(tagged)
                # print(tagged)
                return tagged
    else:
        msg = f'{tagtype} not found.'
        # print(msg)
        return msg

tagtypes = dict(
    dat='DAT',
    tim='TIM',
    loc='LOC',
    wrk='WRK',    
)

# subwordsList, tagsList
nerResults = []
count = 0
for subwords, tags in zip(subwordsList, tagsList):
    # print(f'index: {count}')
    extracted = []
    for tagtype in tagtypes.values():
        # print(tagtype)
        tagged = extract(tagtype, tags, subwords)
        extracted.append(tagged)
        # print()
    nerResults.append(extracted)
    count += 1

In [20]:
# for i in nerResults:
#     print(i)

In [21]:
df = pd.DataFrame(nerResults, columns=['Date', 'Time', 'Location', 'Work'])

In [22]:
df

,Date,Time,Location,Work
0,2013년 2월 일 11,: 30경 부,○○,수행하던 위에서 를 하
1,2012년 11월 28일,TIM not found.,항 1 에서 화물 상,· 하차 에 연결된 를
2,DAT not found.,TIM not found.,LOC not found.,산화물 ( 광물 ) 을
3,2012 . 6 . 7 .,01 : 10경,일본 입항,WRK not found.
4,DAT not found.,TIM not found.,차 정박,WRK not found.
5,##일 15 : 30 을 출항,( 승무원 16,→,"대 , 화물 1 톤 ) 대 고장"
6,201 년 12월 2 일 ( 수 ) 10,TIM not found.,OO 근처에 한 에서 OO 소속,None
7,2012년 12월 10일,TIM not found.,항 60 에서 이 조작,으로 화물이 한쪽으로
8,2015년 12월 1일,22시 30분경,○○ 에 하여 (,에서
9,2015년 12월 10일,TIM not found.,"항 ○○ 원목 에서 원목 , 가",화물차를 오라고


In [17]:
df

,Date,Time,Location,Work
0,2013년 2월 일 11,: 30경 부,○○,수행하던 위에서 를 하
1,2012년 11월 28일,TIM not found.,항 1 에서 화물 상,· 하차 에 연결된 를
2,DAT not found.,TIM not found.,LOC not found.,산화물 ( 광물 ) 을
3,2012 . 6 . 7 .,01 : 10경,일본 입항,WRK not found.
4,DAT not found.,TIM not found.,차 정박,WRK not found.
5,##일 15 : 30 을 출항,( 승무원 16,→,"대 , 화물 1 톤 ) 대 고장"
6,201 년 12월 2 일 ( 수 ) 10,TIM not found.,OO 근처에 한 에서 OO 소속,None
7,2012년 12월 10일,TIM not found.,항 60 에서 이 조작,으로 화물이 한쪽으로
8,2015년 12월 1일,22시 30분경,○○ 에 하여 (,에서
9,2015년 12월 10일,TIM not found.,"항 ○○ 원목 에서 원목 , 가",화물차를 오라고


In [84]:
df['Date'], df['Time'], df['Location'], df['Work'] = df['Date']+'\t', '\t'+df['Time']+'\t', '\t'+df['Location']+'\t', '\t'+df['Work']
df.to_csv('./inference_result.txt', index=False, sep='|')